In [ ]:
from copy import deepcopy
import numpy as np
import time

def bestsolution(state):
    bestsol = np.array([], int).reshape(-1, 9)
    count = len(state) - 1
    while count != -1:
        bestsol = np.insert(bestsol, 0, state[count]['puzzle'], 0)
        count = state[count]['parent']
    return bestsol.reshape(-1, 3, 3)
# checks for the uniqueness of the iteration(it).
def is_unique(state, openstates):
    for s in state:
        if np.array_equal(s['puzzle'], openstates):
            return False
    return True

# number of misplaced tiles
def misplaced_tiles(puzzle, goal):
    mscost = np.sum(puzzle != goal) - 1
    return mscost if mscost > 0 else 0

def coordinates(puzzle):
    pos = np.array(range(9))
    for p, q in enumerate(puzzle):
        pos[q] = p
    return pos

# start of 8 puzzle evaluation, using Misplaced tiles heuristics
def evaluvate_misplaced(puzzle, goal):
    steps = np.array([('up', [0, 1, 2], -3),
                      ('down', [6, 7, 8], 3),
                      ('left', [0, 3, 6], -1),
                      ('right', [2, 5, 8], 1)],
                     dtype=[('move', 'U5'), ('position', list), ('head', int)])
    dtstate = [('puzzle', list), ('parent', int), ('gn', int), ('hn', int)]
    costg = coordinates(goal)

    # initializing the parent, gn and hn, where hn is misplaced_tiles function call
    parent = -1
    gn = 0
    hn = misplaced_tiles(coordinates(puzzle), costg)
    state = np.array([(puzzle, parent, gn, hn)], dtstate)

    # priority queues with position as keys and fn as value.
    dtpriority = [('position', int), ('fn', int)]
    priority = np.array([(0, hn)], dtpriority)

    start_time = time.time()

    while True:
        priority = np.sort(priority, kind='mergesort', order=['fn', 'position'])
        position, fn = priority[0]
        # sort priority queue using merge sort, the first element is picked for exploring.
        priority = np.delete(priority, 0, 0)
        puzzle, parent, gn, hn = state[position]
        puzzle = np.array(puzzle)

        blank = int(np.where(puzzle == 0)[0])
        gn += 1
        for s in steps:
            if blank not in s['position']:
                openstates = deepcopy(puzzle)
                openstates[blank], openstates[blank + s['head']] = openstates[blank + s['head']], openstates[blank]

                if is_unique(state, openstates):
                    hn = misplaced_tiles(coordinates(openstates), costg)
                    # generate and add new state in the list
                    q = np.array([(openstates, position, gn, hn)], dtstate)
                    state = np.append(state, q, 0)
                    # f(n) is the sum of cost to reach node
                    fn = gn + hn

                    q = np.array([(len(state) - 1, fn)], dtpriority)
                    priority = np.append(priority, q, 0)

                    if np.array_equal(openstates, goal):
                        print('The 8 puzzle is solvable \n')
                        return state, len(priority)

                end_time = time.time()
                if (end_time - start_time) > 2:
                    print("The 8 puzzle is unsolvable \n")
                    return state, len(priority)

    return state, len(priority)

# initial state
puzzle = [4, 1, 5, 3, 0, 7, 8, 6, 2]

# goal state
goal = [1, 2, 3, 4, 5, 6, 7, 8, 0]

state, visited = evaluvate_misplaced(puzzle, goal)
bestpath = bestsolution(state)
print(str(bestpath).replace('[', ' ').replace(']', ''))
totalmoves = len(bestpath) - 1
print('\nSteps to reach goal:', totalmoves)
visit = len(state) - visited
print('Total nodes visited: ', visit, "\n")





<ipython-input-3-66fb770b579f>:60: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  blank = int(np.where(puzzle == 0)[0])


The 8 puzzle is unsolvable 

   4 1 5
   3 0 7
   8 6 2

   4 0 5
   3 1 7
   8 6 2

   0 4 5
   3 1 7
   8 6 2

   3 4 5
   0 1 7
   8 6 2

   3 4 5
   1 0 7
   8 6 2

   3 4 5
   1 6 7
   8 0 2

   3 4 5
   1 6 7
   0 8 2

   3 4 5
   0 6 7
   1 8 2

   3 4 5
   6 0 7
   1 8 2

   3 4 5
   6 7 0
   1 8 2

   3 4 5
   6 7 2
   1 8 0

Steps to reach goal: 10
Total nodes visited:  506 

